In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load the data from a JSON file
def load_data(file_path):
    """Load the desired skills and job data from a JSON file."""
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Step 2: Create a recommendation system
def recommend_courses(data, job_title):
    """Recommend AI courses based on the desired skills for a specific job."""
    # Convert the data to a DataFrame
    df = pd.DataFrame(data)

    # Combine the 'desired_skills' column into a single string for vectorization
    df['skills_combined'] = df['desired_skills'].apply(lambda x: ' '.join(x))

    # Use TF-IDF Vectorizer to process the skills data
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['skills_combined'])

    # Find the row corresponding to the given job title
    job_index = df[df['job_title'] == job_title].index
    if job_index.empty:
        return f"Job title '{job_title}' not found in the dataset."

    # Compute cosine similarity
    similarity_scores = cosine_similarity(tfidf_matrix[job_index[0]], tfidf_matrix)

    # Get the top recommendations (excluding the job itself)
    similar_jobs_indices = similarity_scores[0].argsort()[-2::-1]
    recommendations = df.iloc[similar_jobs_indices]

    return recommendations[['job_title', 'desired_skills']]

# Step 3: Example usage
if __name__ == "__main__":
    # Path to the JSON file
    json_file_path = "C:\Users\KIIT0001\Downloads\healthcare_jobs.json"

    # Load the data
    try:
        data = load_data(json_file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {json_file_path}. Please check the file path.")
        exit()

    # Input: Job title to find course recommendations for
    input_job_title = input("Enter the job title: ")

    # Get recommendations
    recommended_courses = recommend_courses(data, input_job_title)

    # Display the recommendations
    if isinstance(recommended_courses, str):
        print(recommended_courses)  # Print error message if job title not found
    else:
        print("\nRecommended Courses based on the job title:")
        print(recommended_courses)

